In [ ]:
import os

import pandas as pd
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
DATA_FILE_DIR = './data/'

START_YEAR, END_YEAR = 2010, 2020

NUM_OF_YEARS = END_YEAR - START_YEAR

NUM_OF_MONTHS = 12

NUM_OF_DAYS = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31,}

In [ ]:
file = nc.Dataset('./data/20110101.nc4')
lat = file.variables['lat'][:].filled()
lon = file.variables['lon'][:].filled()
tmp = file.variables['AvgSurfT_tavg'][0]
num_of_loc = tmp.size - tmp.mask.sum()
file.close()

In [ ]:
LAT = lat.size
LON = lon.size

In [ ]:
def get_tmp(filepath):
    assert os.path.isfile(filepath), '{} does not exist!'.format(filepath)
    
    file = nc.Dataset(filepath)
    temperature = file.variables['AvgSurfT_tavg'][0]
    file.close()
    return temperature.filled(np.nan)

In [ ]:
os.makedirs("./processed_mean_folium", exist_ok=True)
for month in range(1, NUM_OF_MONTHS+1):
    for day in range(1, NUM_OF_DAYS[month]+1):
        date = '{:02d}{:02d}'.format(month, day)
        print(date)
        temps = np.ndarray(shape=(NUM_OF_YEARS, LAT, LON))
        for year in range(START_YEAR, END_YEAR):
            day = str(year) + date
            filepath = DATA_FILE_DIR + day + '.nc4'
            temps[year-START_YEAR] = get_tmp(filepath)
            
        mean_temps = np.nanmin(temps, axis=0)
        mean_temps = mean_temps - 273.15
        mean_temps = pd.DataFrame(mean_temps)
        mean_temps = pd.DataFrame.round(mean_temps, decimals=1)
        mean_temps.to_csv('./processed_mean_folium/{}.csv'.format(date))

In [ ]:
pd.DataFrame(lat).to_csv('./processed_mean_folium/lat.csv')
pd.DataFrame(lon).to_csv('./processed_mean_folium/lon.csv')